In [1]:
#Torchtext není k dispozici pro poslední verzi pytorch, budeme tedy využuívat něco jiného ...


In [2]:
from transformers import Trainer, BertForSequenceClassification, BertTokenizer, EarlyStoppingCallback
from datasets import load_from_disk
import torch
import base
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [3]:
base.reset_seed()

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [5]:
DATASET = "dbpedia"

In [6]:
train = load_from_disk(f"~/data/{DATASET}/train-logits")
eval = load_from_disk(f"~/data/{DATASET}/eval-logits")
test = load_from_disk(f"~/data/{DATASET}/test-logits")

train_aug = load_from_disk(f"~/data/{DATASET}/train-logits-augmented")

In [7]:
tokenizer = BertTokenizer.from_pretrained("gchhablani/bert-base-cased-finetuned-sst2")

In [8]:
train = train.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the train dataset")
eval = eval.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the eval dataset")
test = test.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the test dataset")

train_aug = train_aug.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the augmented dataset")

Tokenizing the train dataset:   0%|          | 0/448000 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
base.reset_seed()

In [ ]:
model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=14)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-base", logging_dir=f"~/logs/{DATASET}/bert-base", batch_size=128, epochs=5)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.536700,0.094711,0.977107,0.977134,0.977107,0.977106
2,0.086400,0.069198,0.982357,0.982351,0.982357,0.982335
3,0.062400,0.064321,0.984009,0.984021,0.984009,0.984008
4,0.052000,0.060336,0.985000,0.984997,0.985000,0.984996
5,0.046300,0.060093,0.985187,0.985182,0.985187,0.985182


TrainOutput(global_step=17500, training_loss=0.1567594970703125, metrics={'train_runtime': 667.5528, 'train_samples_per_second': 3355.54, 'train_steps_per_second': 26.215, 'total_flos': 1673755776000000.0, 'train_loss': 0.1567594970703125, 'epoch': 5.0})

In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
trainer.evaluate(test)

{'eval_loss': 0.06053933501243591,
 'eval_accuracy': 0.9853,
 'eval_precision': 0.9852876597564322,
 'eval_recall': 0.9853000000000002,
 'eval_f1': 0.9852890083123975,
 'eval_runtime': 17.6551,
 'eval_samples_per_second': 3964.869,
 'eval_steps_per_second': 30.983,
 'epoch': 5.0}

In [ ]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bert-base.pth")

In [ ]:
student_model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=14)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-distill", logging_dir=f"~/logs/{DATASET}/bert-distill", remove_unused_columns=False, batch_size=128, epochs=5, temp=5, lambda_param=.5)

In [ ]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.071400,0.340938,0.975732,0.975708,0.975732,0.975704
2,0.234800,0.136575,0.982071,0.982045,0.982071,0.982042
3,0.151200,0.115866,0.983750,0.983764,0.983750,0.983747
4,0.129100,0.106492,0.984821,0.984816,0.984821,0.984814
5,0.119100,0.104360,0.985161,0.985152,0.985161,0.985151


TrainOutput(global_step=17500, training_loss=0.5411379132952009, metrics={'train_runtime': 1132.3516, 'train_samples_per_second': 1978.184, 'train_steps_per_second': 15.455, 'total_flos': 1673755776000000.0, 'train_loss': 0.5411379132952009, 'epoch': 5.0})

In [ ]:
student_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
trainer.evaluate(test)

{'eval_loss': 0.1050909087061882,
 'eval_accuracy': 0.9849,
 'eval_precision': 0.9848861407583568,
 'eval_recall': 0.9849000000000002,
 'eval_f1': 0.9848863992604471,
 'eval_runtime': 19.8826,
 'eval_samples_per_second': 3520.671,
 'eval_steps_per_second': 27.512,
 'epoch': 5.0}

In [ ]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bert-distill.pth")

In [ ]:
base.reset_seed()

In [ ]:
model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=14)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-base-aug", logging_dir=f"~/logs/{DATASET}/bert-base-aug", batch_size=128, epochs=5)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_aug,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.396400,0.070858,0.981795,0.981787,0.981795,0.981784
2,0.095000,0.057410,0.984973,0.984960,0.984973,0.984958
3,0.072600,0.052695,0.986321,0.986320,0.986321,0.986319
4,0.061600,0.052008,0.986518,0.986521,0.986518,0.986517
5,0.055000,0.051981,0.986616,0.986608,0.986616,0.986610


TrainOutput(global_step=34350, training_loss=0.1361017654174627, metrics={'train_runtime': 1755.303, 'train_samples_per_second': 2504.65, 'train_steps_per_second': 19.569, 'total_flos': 3285059539608000.0, 'train_loss': 0.1361017654174627, 'epoch': 5.0})

In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
trainer.evaluate(test)

{'eval_loss': 0.05206460878252983,
 'eval_accuracy': 0.987,
 'eval_precision': 0.9869830786457159,
 'eval_recall': 0.987,
 'eval_f1': 0.9869863816637274,
 'eval_runtime': 12.2043,
 'eval_samples_per_second': 5735.678,
 'eval_steps_per_second': 44.82,
 'epoch': 5.0}

In [ ]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bert-base-aug.pth")

In [ ]:
base.reset_seed()

In [ ]:
student_model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=14)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-distil-aug", logging_dir=f"~/logs/{DATASET}/bert-distil-aug", remove_unused_columns=False, batch_size=128, epochs=5, temp=5, lambda_param=.5)

In [ ]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train_aug,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.316400,0.138778,0.980741,0.980717,0.980741,0.980718
2,0.217100,0.105060,0.984437,0.984420,0.984438,0.984421
3,0.169500,0.094232,0.985696,0.985689,0.985696,0.985690
4,0.147800,0.090227,0.986348,0.986337,0.986348,0.986339
5,0.135900,0.088455,0.986446,0.986433,0.986446,0.986436


TrainOutput(global_step=34350, training_loss=0.3973346151604576, metrics={'train_runtime': 1016.1757, 'train_samples_per_second': 4326.437, 'train_steps_per_second': 33.803, 'total_flos': 3285059539608000.0, 'train_loss': 0.3973346151604576, 'epoch': 5.0})

In [ ]:
student_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
trainer.evaluate(test)

{'eval_loss': 0.08893796801567078,
 'eval_accuracy': 0.9866285714285714,
 'eval_precision': 0.9866166918523136,
 'eval_recall': 0.9866285714285715,
 'eval_f1': 0.9866160083100642,
 'eval_runtime': 13.6123,
 'eval_samples_per_second': 5142.407,
 'eval_steps_per_second': 40.184,
 'epoch': 5.0}

In [ ]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bert-distil-aug.pth")